In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

from __future__ import print_function
import os, sys
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
print("hello")

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
hello


In [3]:
sns.set_style("whitegrid")
sns.set_context("paper")
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.unicode'] = True
plt.rcParams['image.cmap'] = 'viridis'

# Add cornet and Theano to path
sys.path.insert(0, '../')
sys.path.insert(0, '/home/poole/Theano_clean')

# Output figure directory
figure_dir = "../writeup/figures"
results_dir = "../results"
if not os.path.exists(figure_dir):
    raise ValueError("Figure directory does not exist. Create it or change figure_dir above.")
if not os.path.exists(results_dir):
    raise ValueError("Results directory does not exist. Create it or change results_dir above.")

In [4]:
#choice of non linearity and 1st and 2nd derivatives
from scipy.stats import hypsecant
def sech(x):
    return hypsecant.pdf(x) * np.pi

phi = np.tanh
dphi = lambda x: sech(x)**2
d2phi = lambda x: 2 * (np.tanh(x)**3 - np.tanh(x))


In [5]:
from cornet.constants import *
print("Weight sigmas (n=%d):" % nw, weight_sigmas)
print("Bias sigmas (n=%d):" % nb, bias_sigmas)

Weight sigmas (n=41): [1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7
 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5
 4.6 4.7 4.8 4.9 5. ]
Bias sigmas (n=41): [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5
 3.6 3.7 3.8 3.9 4. ]


In [6]:
from cornet.theory import qmap, q_fixed_point

In [7]:
# Range of squared lengths for q map plot (panel A)
nq = 100
qmax = 15
qrange = np.linspace(0, qmax, nq)

# Number of iterations for the dynamics of convergence plot (panel B)
nt = 20

# Maximum number of iterations when computing convergence time (panel D)
nconverge = 25

qconverge = np.linspace(0, qmax, nconverge)
qmaps = np.zeros((nw, nb, nq, nt))
qstars = np.zeros((nw, nb))
tstars = np.zeros((nw, nb, nconverge))
qmaps[..., 0] = qrange[None, None, None, :]

for widx, weight_sigma in enumerate(tqdm(weight_sigmas)):
    for bidx, bias_sigma in enumerate(bias_sigmas):
        # Compute fixed points
        _, qstars[widx, bidx]  =  q_fixed_point(weight_sigma, bias_sigma, nonlinearity, qinit=3.0, tol_frac=0.01)
        # Iterate over initial norms, computing convergence time
        for tidx, qinit in enumerate(qconverge):
            tstars[widx, bidx, tidx], _  =  q_fixed_point(weight_sigma, bias_sigma, nonlinearity, qinit=qinit, tol_frac=0.01)
        # Dynamics of convergence of q map
        for t in range(1, nt):
            qmaps[widx, bidx, :, t] = qmap(qmaps[widx, bidx, :, t - 1], weight_sigma, bias_sigma, nonlinearity)

  0%|          | 0/41 [00:00<?, ?it/s]..\cornet\theory.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  frac_err = (np.array(qs) - q)**2 / (1e-9 + q**2)
100%|██████████| 41/41 [02:04<00:00,  3.04s/it]


In [8]:
from cornet.net import RandNet
n_hidden_layers = 1
n_hidden_units = 1000
din = 1000
# Flip flag here chooses whether activations or linear layer is first
## removed: nonlinearity=nonlinearity argument
net = RandNet(din, n_hidden_units, n_hidden_layers, flip=False)

AsTensorError: ('Cannot convert KerasTensor(type_spec=TensorSpec(shape=(None, 1000), dtype=tf.float32, name=\'l_d0_input\'), name=\'l_d0_input\', description="created by layer \'l_d0_input\'") to TensorType', <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>)